# Semantic Kernel 與 OpenBnB MCP 伺服器整合

此筆記本展示如何使用 Semantic Kernel 與實際的 OpenBnB MCP 伺服器結合，透過 MCPStdioPlugin 搜尋真實的 Airbnb 住宿。對於 LLM 存取，使用的是 Microsoft Foundry。要設定您的環境變數，您可以參考 [Setup Lesson ](/00-course-setup/README.md)


## 匯入所需套件


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## 建立 MCP 外掛連線

我們將使用 MCPStdioPlugin 連接到 [OpenBnB MCP 伺服器](https://github.com/openbnb-org/mcp-server-airbnb)。此伺服器透過 @openbnb/mcp-server-airbnb 套件提供 Airbnb 搜尋功能。


## 建立用戶端

在本範例中，我們將使用 Microsoft Foundry 作為我們的 LLM 存取。請確保您的環境變數已正確設定。


## Environment Configuration

Configure Azure OpenAI settings. Make sure you have the following environment variables set:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## 理解 OpenBnB MCP 整合

此筆記本連接到提供實際 Airbnb 搜尋功能的**真實 OpenBnB MCP 伺服器**。

### 運作方式：

1. **MCPStdioPlugin**：使用標準輸入/輸出與 MCP 伺服器通訊  
2. **真實 NPM 套件**：透過 npx 下載並執行 `@openbnb/mcp-server-airbnb`  
3. **即時資料**：從 Airbnb API 回傳真實的房源資料  
4. **函式探索**：代理會自動從 MCP 伺服器發現可用函式  

### 可用函式：

OpenBnB MCP 伺服器通常提供：  
- **search_listings** - 根據位置和條件搜尋 Airbnb 房源  
- **get_listing_details** - 取得特定房源的詳細資訊  
- **check_availability** - 檢查特定日期的可用性  
- **get_reviews** - 取得房源評論  
- **get_host_info** - 取得房東資訊  

### 前置需求：

- 系統已安裝 **Node.js**  
- 具有網路連線，可下載 MCP 伺服器套件  
- 可使用 **NPX**（隨 Node.js 附帶）  

### 測試連線：

您可以手動執行以下指令測試 MCP 伺服器：  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
這將下載並啟動 OpenBnB MCP 伺服器，然後 Semantic Kernel 會連接它以取得真實 Airbnb 資料。


## 使用 OpenBnB MCP 伺服器運行代理程式

現在我們將運行連接到 OpenBnB MCP 伺服器的 AI 代理程式，以搜尋斯德哥爾摩的真實 Airbnb 住宿，入住人數為 2 成人和 1 位孩童。隨時可以更改 `user_inputs` 列表以修改搜尋條件。


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Summary
恭喜！您已成功建立一個整合現實住宿搜尋的 AI 代理，使用 Model Context Protocol (MCP)：

## Technologies Used:
- Semantic Kernel - 用於使用 Azure OpenAI 建立智慧代理
- Microsoft Foundry - 提供大型語言模型功能及聊天完成
- MCP (Model Context Protocol) - 用於標準化工具整合
- OpenBnB MCP Server - 用於真實 Airbnb 搜尋功能
- Node.js/NPX - 用於執行外部 MCP 伺服器

## What You've Learned:
- MCP Integration：將 Semantic Kernel 代理連接到外部 MCP 伺服器
- Real-time Data Access：透過即時 API 搜尋真實 Airbnb 物件
- Protocol Communication：使用 stdio 通訊於代理與 MCP 伺服器之間
- Function Discovery：自動發現 MCP 伺服器可用的功能
- Streaming Responses：即時擷取並記錄函式呼叫
- HTML Rendering：以樣式化表格及互動展示格式化代理回應

## Next Steps:
- 整合額外 MCP 伺服器（天氣、航班、餐廳）
- 建立結合 MCP 與 A2A 協定的多代理系統
- 為您自己的資料來源創建自訂 MCP 伺服器
- 實作跨會話的持久對話記憶
- 將代理部署至 Azure Functions 並搭配 MCP 伺服器協調
- 新增使用者身份驗證及訂房功能


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們力求準確，但請注意自動翻譯可能包含錯誤或不準確之處。原始文件及其母語版本應視為權威依據。對於重要資訊，建議採用專業人工翻譯。本公司不對因使用此翻譯而產生之任何誤解或誤譯負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
